In [0]:
# Basic init & import
%pylab inline
import networkx as nx
import numpy as np

In [0]:
# Store dataset or load a previous one?
newDataset=False
dataFolder='data/'

import pickle
file = open(dataFolder+'dataset.txt', 'r')
data = pickle.load(file)
PPI=data['PPI']
patients=data['patients']
phenotypes=data['phenotypes']
pathwaysNum=data['pathwaysNum']
genesNum=data['genesNum']
connProbability=data['connProbability']
connNeighboors=data['connNeighboors']
connBetweenPathways=data['connBetweenPathways']
patientsNum=data['patientsNum']
mutationProb=data['mutationProb']
file.close()

In [0]:
# Diffusion of the mutations according to PPI topology
PPIAdjacencyMatrix=nx.to_numpy_matrix(PPI)

def mutationProfileDiffusion(mutationProfile,PPIAdjacencyMatrix,diffusionFactor):
    tmp=np.array(PPIAdjacencyMatrix.sum(axis=0))
    tmp[tmp==0]=1
    A=PPIAdjacencyMatrix*np.diagflat(1./tmp)
    X1=mutationProfile
    X2=diffusionFactor*X1*A+(1-diffusionFactor)*mutationProfile
    while norm(X2-X1)>10e-6:
        X1=X2
        X2=diffusionFactor*X1*A+(1-diffusionFactor)*mutationProfile
    return X2

In [0]:
diffusionFactor=0.8
# Usefull functions
def assignMutation(PPI,mutationProfile):
	for gene in range(0,len(PPI)):
		PPI.node[gene]['val']=mutationProfile[gene]
	return PPI

def extractMutationProfile(PPI):
    return [float(PPI.node[gene]['val']) for gene in PPI.nodes()]

patientsDiffused=mutationProfileDiffusion(patients,PPIAdjacencyMatrix,diffusionFactor)

In [0]:
# Visualization of the patients networks after diffusion
#import matplotlib.pyplot as plt
plt.figure(1,figsize=(12,12))
pos=nx.graphviz_layout(PPI,prog="neato")
for pn in range(0,patientsNum):
    plt.subplot(10, 6, pn+1)
    nx.draw(PPI,pos,with_labels=False,node_size=30,node_color=np.array(patientsDiffused[pn,:]),cmap = plt.cm.Blues)
plt.show()

In [0]:
# Store diffused dataset?
newDataset=True

if newDataset:
    file = open(dataFolder+'datasetDiffused.txt', 'w')
    data = {'PPI': PPI, 'patients' : patients, 'patientsDiffused' : patientsDiffused, 'phenotypes': phenotypes, 'pathwaysNum': pathwaysNum,'genesNum': genesNum,'connProbability': connProbability,'connNeighboors': connNeighboors,'connBetweenPathways': connBetweenPathways,'patientsNum': patientsNum,'mutationProb': mutationProb,'diffusionFactor': diffusionFactor}
    pathwaysNum,genesNum,connProbability,connNeighboors,connBetweenPathways,patientsNum,mutationProb
    pickle.dump(data, file)
file.close()